## Loading all the libraries


In [99]:
import pandas as pd
import numpy as np
import pickle
import math
import time

## Loading and cleaning the data.
We take the words from Shakespears play, Hamlet. 

In [100]:
# loading the data
data = pd.read_csv('sample_data/Shakespeare_data.csv')

# getting rid of unwanted rows.
df = data.dropna (subset = ['Player', 'ActSceneLine'])

# Getting the words from the play Hamlet.
new_data = df.loc[data['Play'] == 'Hamlet',['PlayerLine']]
lines = new_data['PlayerLine'].values.tolist()

# removing all the punctuations. Excluding 'period'
punc = '''!()-[]{};:'"\,<>/?@#$%^&*_~""'''
for i in range(len(lines)):
    temp_line = ""
    for item in lines[i]:
        if item not in punc:
            temp_line += item
    
    lines[i] = temp_line

# forming a list of words.
words = ""
for i in range(len(lines)):
    words = words + lines[i].lower()+ " "        
words = words.split(" ")

# collecting all the unique words in the play Hamlet!
unique = []
for item in words:
    if item not in unique:
        unique.append(item)
unique.remove("")



## Finding the parameters of Hidden Markov Model using the Baum Welch algorithm.

The parameters are being saved in a pickle file named parameters.p

In [103]:
# We will use the Baum-welch algorithm! This is a lot similar to Expectation Maximization algorithm done in the first project.
# The algorithms have been referred and understood from the textbook: 
# Also to avoid many underflow errors a scaled version of the parameters are used. 

def normalized(a):
    return a/sum(a)


def train(words, latent_variables):
    n = len(words)
    # start time
    begin = time.time()

    # To train the model, we need to initiale the hidden markov model with initial parameters. 
    # We train the model to update the parameters to get optimal ones. 
    
    # We start the algorithm with initial parameters. 
    temp1 = np.random.uniform(10,50,latent_variables)
    initial = normalized(temp1)
    old_initial = np.empty([latent_variables])
    
    # Initial transmission probabilities. This is again done with Dirichlet distribution.
    # see this as from rows to columns
    ini_trans = np.empty([latent_variables,latent_variables])
    for i in range(latent_variables):
        temp2 = np.random.uniform(1,10,latent_variables)
        ini_trans[i] = normalized(temp2)
    old_ini_trans = np.empty([latent_variables,latent_variables])

    # Initial emmision probabilities.
    ini_emi = np.empty([latent_variables,n])
    old_ini_emi = np.empty([latent_variables,n])

    for i in range(latent_variables):
        temp3 = np.random.uniform(1,6,n)
        ini_emi[i] = normalized(temp3)

    count = 0

    while not np.array_equal(old_initial, initial):

        if count > 150:
            break

        count +=1
        #print("Iteration:", count)

        # For the algorithm to run we find forward parameters of the algorithm, backward part, gamma and etc..

        # Finding scaled alpha, this is the forward part of the algorithm.
        alpha = np.empty([n,latent_variables])
        normalized_alpha = np.empty([n,latent_variables])
        c = np.empty([n])
        for i in range(latent_variables):
            alpha[0][i] = initial[i]*ini_emi[i][0]

        c[0] = sum(alpha[0])
        if math.isnan(c[0]) == True:
            print("Try again with new initial parameters")
            break
        #print(c[0])
        normalized_alpha[0] = normalized(alpha[0])

        # Finding scaled alpha recursively. 
        for i in range(1,n):
            temp4 = np.empty([latent_variables])
            for j in range(latent_variables):
                summ = 0
                for z in range(latent_variables):
                    summ += normalized_alpha[i-1][z] * ini_trans[z][j]
                
                temp4[j] = ini_emi[j][i] * summ

            c[i] = sum(temp4)
            normalized_alpha[i] = temp4/c[i]

        
        # Finding backward parameters
        beta = np.empty([n,latent_variables])
        normalized_beta = np.empty([n,latent_variables])
        # initializing.. 
        beta[n-1] = np.ones([latent_variables])
        normalized_beta[n-1] = np.ones([latent_variables])
        for i in range(1,n):
            temp = np.empty([latent_variables])
            for j in range(latent_variables):
                summ1 = 0
                for z in range(latent_variables):
                    summ1 += normalized_beta[n-i][z]*ini_emi[z][n-i]*ini_trans[j][z]

                temp[j] = summ1
            
            normalized_beta[n-1-i] = temp/c[n-i]


        # Gamma parameter in algorithm
        gamma = np.empty([n,latent_variables])
        for i in range(n):
            for j in range(latent_variables):
                gamma[i][j] = normalized_alpha[i][j] * normalized_beta[i][j]


        # Xi parameter in algorithm
        xi = np.empty([n,latent_variables,latent_variables])

        for i in range(1,n):
            for j in range(latent_variables):  # for z_n-1 variable
                for z in range(latent_variables): # for z_n variable
                    xi[i][j][z] = c[i]*normalized_alpha[i-1][j] * ini_emi[z][i] *ini_trans[j][z] * normalized_beta[i][z]


        # Saving old parameters!
        old_initial = np.copy(initial)
        old_ini_trans = np.copy(ini_trans)
        old_ini_emi = np.copy(ini_emi)

        # Now we will use these parameter to update the initial parameters!


        # updated initial latent variable probabilities. 
        for i in range(latent_variables):
            initial[i] = gamma[0][i]/sum(gamma[0])
        
        #print(" Initial probability: ",sum(initial))


        # updated transition probabilities. 
        for i in range(latent_variables):
            deno = 0
            for l in range(latent_variables):
                for k in range(1,n):
                    deno += xi[k][i][l]
            
            for j in range(latent_variables):
                nume = 0
                for z in range(1,n):
                    nume += xi[z][i][j]
                
                ini_trans[i][j] = nume/deno

            #print("probability of transmission: ", sum(ini_trans[i]))


        # updating emission prbabilities.
        for i in range(latent_variables):
            denom = sum(gamma[:,i])
            for j in range(n):
                ini_emi[i][j] = gamma[j][i]/denom

        #print(np.array_equal(old_initial, initial))
    # calculating end time.
    end = time.time()

    if count < 100:
        print("Convergence of parameters achieved in ", count, "iterations and in time: ", end-begin, "seconds")            
    parameters = (initial, ini_trans, ini_emi,latent_variables)
    
    # Pickle helps save the model in a seperate file that can be accessed later. 
    pickle.dump(parameters, open('sample_data/pickle/parameters.p', 'wb'))
        
# training with 7 latent variables. One can think of them as different parts of speech in english, like noun, verb etc
train(unique,7)


Convergence of parameters achieved in  13 iterations and in time:  29.666426420211792 seconds


## The text generation and text prediction functions

In [104]:
# The word generating function! uses the trained model.
# paramaters for both functions are the list of unique words in the play.

def generate(words):
    # loading the trained data
    parameters = pickle.load(open('sample_data/pickle/parameters.p', 'rb'))

    lat_var = parameters[3]
    initial = parameters[0]
    ini_trans = parameters[1]
    ini_emi = parameters[2]

    n = int(input("Enter number of words to generate: "))
    
    # choosing a random initial latent variable in accordance with initial probabilities
    latent = np.random.choice(list(range(lat_var)),p = initial)

    sentence = ""
    for i in range(n):
        # chosing random word from latent variable in accordance with model emission parameters.
        word = np.random.choice(words, p = ini_emi[latent])
        if sentence == "":
            word = word.capitalize()
        elif sentence[-2] == ".":
            word = word.capitalize()
        sentence += word + " "
        # choosing next latent variable in accordance with model transision probabilities.
        latent = np.random.choice(list(range(lat_var)), p = ini_trans[latent])

    print(sentence)



# This is the text prediction function. uses viterbi algorithm. 
# for the input, we have to use words in the argument of the function. 

def text_prediction(words):

    # loading the trained data
    parameters = pickle.load(open('sample_data/pickle/parameters.p', 'rb'))

    no_late_var = parameters[3]
    initial = parameters[0]
    ini_trans = parameters[1]
    ini_emi = parameters[2]


    sentence = input("Write a sentence: ")
    #sentence = "whos there nay answer and long live king quiet guard"
    n = int(input("Enter no of words to predict: "))

    # spliting the sentence into a list of words
    sentence_lst = sentence.split(" ")

    probabilities = np.empty([no_late_var, len(sentence_lst)])

    x = np.empty([len(sentence_lst)])

    # Figuring out best possible latent variable for the first word. 
    for i in range(no_late_var):
        probabilities[i][0] = initial[i] * ini_emi[i][uniq.index(sentence_lst[0])]
    # x stores the best latent variable!
    x[0] = np.argmax(probabilities[:,0])
    
    #Figuring out best subsequent sequence of latent variables. (We find the highest probable latent variable at each step)
    for z in range(1,len(sentence_lst)):
        for i in range(no_late_var):
            temp = np.empty([no_late_var])
            for j in range(no_late_var):
                temp[j] = probabilities[j][z-1] * ini_trans[j][i] * ini_emi[i][uniq.index(sentence_lst[z])]

            index = np.argmax(temp)
            
            probabilities[i][z] = temp[index]
        # Record the best possible latent variable
        x[z] = np.argmax(probabilities[:,z])


    #The last latent variable is x[-1], from here we follow the code for the generate function. 
    latent = int(x[-1])
    predict = " "
    for i in range(n):
        word = np.random.choice(words, p = ini_emi[latent])
        if i >0 and predict[-2] == ".":
            word = word.capitalize()
        predict += word + " "
        latent = np.random.choice(list(range(no_late_var)), p = ini_trans[latent])

    print("Most probable words to follow are: ",predict)







## Results

In [105]:
generate(unique)

Enter number of words to generate: 50
Too canker singeing smells gis dew plautus sterling. Earth. Selfslaughter rheum beaten littlest relish mouth forgone wag provincial natures. Forbid distrust beneath. Residence distempered. Dearly cellarage liferendering spring scarce courteous sigh loses warlike laertes purpose extremity. Rhymed. Roses carters. Entertainment womens attended. Melancholy stayd roses rheum eight pleasing charity normandy 


In [106]:
generate(unique)

Enter number of words to generate: 20
Craft impious unyoke. Afflict seat sick easy tediousness went ungracious attractive. Doors knavish acres orbed sliver necessary through distracted unmannerly. 


In [107]:
generate(unique)

Enter number of words to generate: 15
Laertes. Hilts unite adoption certainty breaking way. Brainish twice wars fixd natural canon laertes tender 


In [108]:
text_prediction(unique)

Write a sentence: long live the
Enter no of words to predict: 5
Most probable words to follow are:   prevent disasters crafts deprived physic 


In [110]:
text_prediction(unique)

Write a sentence: i am going back to
Enter no of words to predict: 4
Most probable words to follow are:   went erring hundred adventurous 


In [111]:
text_prediction(unique)

Write a sentence: stars in the night will
Enter no of words to predict: 4
Most probable words to follow are:   satisfied plunge primal earth 
